In [1]:
# consider the subscription, redeemers, and dcm clickers
import pandas as pd
import numpy as np
import datetime
import os
import paramiko
import gc

In [2]:
'''
host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

remote_file_1_dcm_fb_clickers='/mnt/drv5/lr_biglots_data/download_logs/clicks/CL_BigLots_dcm_account7252_click_20191021_20191107_fboffers_BL_20191206_053923_0000.tsv.gz'
remote_file_2_Q3_mapping="/mnt/drv5/lr_biglots_data/download_logs/others/CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_0000.psv.gz"

remote_file_list=[remote_file_1_dcm_fb_clickers,remote_file_2_Q3_mapping]

for remote_file in remote_file_list:
    local_path="./"+os.path.basename(remote_file)
    sftp.get(remote_file,local_path)
sftp.close()
transport.close()
'''

'\nhost = "64.237.51.251" #hard-coded\nport = 22\ntransport = paramiko.Transport((host, port))\n\npassword = "jian@juba2017" #hard-coded\nusername = "jian" #hard-coded\ntransport.connect(username = username, password = password)\nsftp = paramiko.SFTPClient.from_transport(transport)\n\nremote_file_1_dcm_fb_clickers=\'/mnt/drv5/lr_biglots_data/download_logs/clicks/CL_BigLots_dcm_account7252_click_20191021_20191107_fboffers_BL_20191206_053923_0000.tsv.gz\'\nremote_file_2_Q3_mapping="/mnt/drv5/lr_biglots_data/download_logs/others/CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_0000.psv.gz"\n\nremote_file_list=[remote_file_1_dcm_fb_clickers,remote_file_2_Q3_mapping]\n\nfor remote_file in remote_file_list:\n    local_path="./"+os.path.basename(remote_file)\n    sftp.get(remote_file,local_path)\nsftp.close()\ntransport.close()\n'

In [3]:
df_clickers=pd.read_csv("./CL_BigLots_dcm_account7252_click_20191021_20191107_fboffers_BL_20191206_053923_0000.tsv.gz",
                       compression="gzip",sep="\t",dtype=str,
                        usecols=['Customer_Link','Event Time','Advertiser ID','Campaign ID','Ad ID','Site ID (DCM)','Placement ID'])
print(df_clickers.shape)
df_clickers=df_clickers[df_clickers['Customer_Link']!="UNMATCHED"]
print(df_clickers.shape)
df_clickers.shape,df_clickers['Customer_Link'].nunique()

(201624, 7)
(40660, 7)


((40660, 7), 31219)

In [4]:
df_clickers.head(2)

,Customer_Link,Event Time,Advertiser ID,Campaign ID,Ad ID,Site ID (DCM),Placement ID
1,XY146898E-Q2rTzSJCtYbthhYTjeBDTPpCEEiCyJil8I-Bahw,1572895050375612,8095847,23030980,454197183,2710167,258481638
19,XY1468M6qU6CSXTXFx5t1ON3qyJOOEKbzWV1KEQX-fpvjsUzU,1572687338036744,8095847,23030980,454273339,2710167,258073193


In [5]:
df_mapping=pd.read_csv("./CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_0000.psv.gz",
                      dtype=str,sep="|",compression="gzip",usecols=['Customer_Link','customer_id_hashed']).drop_duplicates()
df_mapping.shape

(24078139, 2)

In [6]:
df_clickers=pd.merge(df_clickers,df_mapping,on="Customer_Link",how="left")
print(df_clickers.shape)

list_customer_ids_clickers=df_clickers['customer_id_hashed'].unique().tolist()
df_click_ids=pd.DataFrame({"customer_id_hashed":list_customer_ids_clickers})
df_click_ids=df_click_ids[~pd.isnull(df_click_ids['customer_id_hashed'])]

(40945, 8)


In [7]:
df_clickers.head(5)

,Customer_Link,Event Time,Advertiser ID,Campaign ID,Ad ID,Site ID (DCM),Placement ID,customer_id_hashed
0,XY146898E-Q2rTzSJCtYbthhYTjeBDTPpCEEiCyJil8I-Bahw,1572895050375612,8095847,23030980,454197183,2710167,258481638,0696d2f9ca06589324f76bbbf6f359e87f292db87a5253...
1,XY1468M6qU6CSXTXFx5t1ON3qyJOOEKbzWV1KEQX-fpvjsUzU,1572687338036744,8095847,23030980,454273339,2710167,258073193,NaN
2,XY1468yrG3_vuCdJJGFFfy3syaKtX2dr1Mk3or5fV-cVdhlEk,1572687121864585,8095847,23030980,454273339,2710167,258073193,NaN
3,XY1468ltKlOYOmPbpSMRTNUrintJAKD8sDgXFsLqOvK84odls,1572688700853642,8095847,23030980,454273339,2710167,258073193,NaN
4,XY1468EnRUYaMj6nqTZwqbWE7Y0cmp_TZmfBE8i5eeEx-HvOI,1572688903301130,8095847,23030980,454273339,2710167,258073193,NaN


In [8]:
df_redeemers=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Slice_ids_FB_Rewards_offers/ID_level_report_of_redeemers/hash_full_file_redeemers_1201.zip",
                        dtype=str,sep="|",compression="zip")
print(df_redeemers.shape,df_redeemers['hashed_customer_num'].nunique(),df_redeemers['hashed_email_address'].nunique())
df_redeemers.head(2)

(262870, 2) 262870 262821


,hashed_customer_num,hashed_email_address
0,41448b7946e03d434fb9efe139eff9d64d0a97b58ca15b...,2b575b29eb1a236c2e43ea4b8d07c09391773e401535e9...
1,dad71dc369858e4d71755b3b88701a3dea4de62eef8ac7...,28bb15e7436c23796ebf07a42e3787bfabf1ed8ad9eb3a...


In [9]:
df_FB_id_email_all=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Slice_ids_FB_Rewards_offers/BL_all_ids_shopped_in_18_or_signed_in_12_with_emails_JL_2019-10-17.csv",dtype=str)
df_FB_id_email_all.shape

(21505499, 2)

In [10]:
print(df_FB_id_email_all['email_address_hash'].nunique())

21503336


In [11]:
df_test=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Slice_ids_FB_Rewards_offers/BL_FB_5_offer_test_emails_JL_2019-10-17.csv",dtype=str)
df_control=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Slice_ids_FB_Rewards_offers/BL_FB_5_offer_control_emails_JL_2019-10-17.csv",dtype=str)

df_test['Group']="test"
df_control['Group']="control"

df_email_group=df_test.append(df_control)
del df_test
del df_control
print(df_email_group.shape)

df_email_group.head(2)

(21503357, 2)


,Email,Group
0,089bc373eda16a771e023d87ee368fb961c4ff9b36c7d6...,test
1,065edf59d0ed26bea352be5f44fd7a6f688a0fa28cddb3...,test


In [12]:
df_qc_duplicate=df_email_group.groupby("Email")['Group'].count().to_frame().sort_values("Group",ascending=False)

# 2 the max
print(df_qc_duplicate[df_qc_duplicate['Group']==2].shape )
df_qc_duplicate=df_qc_duplicate.reset_index()
Emails_list_21_in_both=df_qc_duplicate[df_qc_duplicate['Group']==2]['Email'].tolist()
df_email_group[df_email_group['Email'].isin(Emails_list_21_in_both)].sort_values(["Email","Group"])

(21, 1)


,Email,Group
95446,1152cb7e6f7bba4aea1a4a9543cefeb4e5f5fccd92a0c5...,control
2118993,1152cb7e6f7bba4aea1a4a9543cefeb4e5f5fccd92a0c5...,test
15843,12d0aded1a1810dd08e60ea3bb1d0b0d46b1a2c224a1e0...,control
14076876,12d0aded1a1810dd08e60ea3bb1d0b0d46b1a2c224a1e0...,test
30196,2b5aadd1188c83cfcd23d14e2578a1af92e41a7198ca1f...,control
19978211,2b5aadd1188c83cfcd23d14e2578a1af92e41a7198ca1f...,test
25126,33c21258cdfeaae52ea857b0af6a72975fb4963ee9f065...,control
20562300,33c21258cdfeaae52ea857b0af6a72975fb4963ee9f065...,test
85183,3f0373fa1412044927d83a5bd105abc584ee8a423607fc...,control
20870886,3f0373fa1412044927d83a5bd105abc584ee8a423607fc...,test


In [13]:
# The 21 emails are excluded from both test and control

del df_qc_duplicate
df_email_group=df_email_group[~df_email_group['Email'].isin(Emails_list_21_in_both)]
df_email_group=df_email_group.rename(columns={"Email":"email_address_hash"})

print(datetime.datetime.now(),df_FB_id_email_all.shape)
df_FB_id_email_all=pd.merge(df_FB_id_email_all,df_email_group,on="email_address_hash",how="left")
print(datetime.datetime.now(),df_FB_id_email_all.shape)

2019-12-11 17:20:50.699275 (21505499, 2)
2019-12-11 17:22:13.478828 (21505499, 3)


In [14]:
print(df_FB_id_email_all.shape,df_FB_id_email_all['customer_id_hashed'].nunique())

(21505499, 3) 21505499


In [15]:
print(df_FB_id_email_all.shape,df_FB_id_email_all['customer_id_hashed'].nunique())
print(df_click_ids.shape,df_click_ids['customer_id_hashed'].nunique())

df_FB_id_email_all=df_FB_id_email_all.append(df_click_ids).drop_duplicates("customer_id_hashed")
print(df_FB_id_email_all.shape,df_FB_id_email_all['customer_id_hashed'].nunique())

(21505499, 3) 21505499
(5535, 1) 5535


/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


(21506042, 3) 21506042


In [16]:
df_FB_id_email_all.tail(5)

,Group,customer_id_hashed,email_address_hash
5487,NaN,7bb07312e88de27dfb4478c91d33395fc638adf08a18d6...,NaN
5491,NaN,25e8136036baf51fa2d85723229369e39786e4f4ec330c...,NaN
5510,NaN,bca637cff5a94103cde1759a9bf0552dd8393ec2e81ce7...,NaN
5513,NaN,2a811da19f54ff37a19d0261fd4035f011cb1f83ba9621...,NaN
5514,NaN,ff507b616b8441b7740a90fe476538114241b4a935cfd4...,NaN


In [17]:
df_FB_id_email_all[pd.isnull(df_FB_id_email_all['email_address_hash'])].shape

# BL matched ids that not in the uploaded facebook audiences

(543, 3)

In [18]:
df_FB_id_email_all['Group']=df_FB_id_email_all['Group'].fillna("clicker_not_uploaded")
df_FB_id_email_all['email_address_hash']=df_FB_id_email_all['email_address_hash'].fillna("nan")

In [19]:
list_redeamer_id=df_redeemers['hashed_customer_num'].tolist()
list_redeamer_email=df_redeemers['hashed_email_address'].tolist()
df_FB_id_email_all['redemption']=np.where((df_FB_id_email_all['customer_id_hashed'].isin(list_redeamer_id) | df_FB_id_email_all['email_address_hash'].isin(list_redeamer_id)),"redeemers","others" )

In [20]:
df_FB_id_email_all.shape

(21506042, 4)

In [21]:
df_redeemers_others=df_redeemers[~df_redeemers['hashed_customer_num'].isin(df_FB_id_email_all['customer_id_hashed'].tolist())]
df_redeemers_others=df_redeemers_others[~df_redeemers_others['hashed_email_address'].isin(df_FB_id_email_all['email_address_hash'].tolist())]
df_redeemers_others['Group']="redeemers_not_uploaded"
df_redeemers_others['redemption']="redeemers"

df_redeemers_others=df_redeemers_others.rename(columns={"hashed_customer_num":"customer_id_hashed","hashed_email_address":"email_address_hash"})
df_FB_id_email_all=df_FB_id_email_all.append(df_redeemers_others)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [22]:
df_FB_id_email_all.shape

(21527626, 4)

In [23]:
df_FB_id_email_all.head(2)

,Group,customer_id_hashed,email_address_hash,redemption
0,test,9b7f9a1ba51dbae2a393c273ce8c9aa9bf610dc03410fd...,089bc373eda16a771e023d87ee368fb961c4ff9b36c7d6...,others
1,test,c2c5bd862113c4e77c918dd22a25744f2e1224ea63b166...,065edf59d0ed26bea352be5f44fd7a6f688a0fa28cddb3...,others


In [24]:
# del df_FB_id_email_all['hashed_customer_num']
# del df_FB_id_email_all['hashed_email_address']
gc.collect()

print(df_FB_id_email_all.shape,df_FB_id_email_all['customer_id_hashed'].nunique(),df_FB_id_email_all['email_address_hash'].nunique())


(21527626, 4) 21527626 21524914


In [25]:
df_FB_id_email_all=df_FB_id_email_all.drop_duplicates()
df_FB_id_email_all.shape

(21527626, 4)

In [26]:
df_FB_id_email_all.head(5)

,Group,customer_id_hashed,email_address_hash,redemption
0,test,9b7f9a1ba51dbae2a393c273ce8c9aa9bf610dc03410fd...,089bc373eda16a771e023d87ee368fb961c4ff9b36c7d6...,others
1,test,c2c5bd862113c4e77c918dd22a25744f2e1224ea63b166...,065edf59d0ed26bea352be5f44fd7a6f688a0fa28cddb3...,others
2,test,d1d5e857a51c2d795f07f08d600c1fda1eeb85388d02ef...,310e51ebfe35397b827828c4fe40faf4f9b0336aecf2f7...,others
3,test,a5879872bfdf0b1ced681739a58dec90b19cd96b21456d...,790b7ad5b7d7c1ecfb974a6174e1e814be93ab26f4e459...,others
4,test,b9792370f5f7e061b85bfa1bc0fbcdd7c0d7b83ce153b8...,25ec4dd616563a3a675c500bedf4a590dfe43d27b636e3...,others


In [27]:
# Add clicker col
df_FB_id_email_all['FB_clicker']=np.where(df_FB_id_email_all['customer_id_hashed'].isin(list_customer_ids_clickers),"clickers","non_clickers")


In [28]:
df_test=df_FB_id_email_all[df_FB_id_email_all['redemption']=="redeemers"]
df_test.shape

(262852, 5)

In [29]:
# unsubsribers 
df_unsub_emails=pd.read_csv("/home/jian/BigLots/unsubscribe/Unsub_JH_Final20191105Hashed.zip",
                              compression="zip",dtype=str)
list_unsub_emails=df_unsub_emails['hashed_email_address'].unique().tolist()
list_unsub_ids=df_unsub_emails['hashed_customer_num'].unique().tolist()

print(df_unsub_emails.shape,len(list_unsub_ids),len(list_unsub_emails))



(2771375, 2) 2771375 2771350


In [ ]:
df_FB_id_email_all['unsubscription']=np.where((df_FB_id_email_all['customer_id_hashed'].isin(list_unsub_ids) | df_FB_id_email_all['email_address_hash'].isin(list_unsub_emails)),"unsubscribed","default_subscribed")


In [ ]:
# Sales

# POS data range
file_week_1="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-10-26/MediaStormDailySales20191029-111155-920.txt"
file_week_2="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-02/MediaStormDailySales20191105-111212-139.txt"
file_week_3="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt"

POS_file_list=[file_week_1,file_week_2,file_week_3]

df_POS_all=pd.DataFrame()

for file in POS_file_list:
    df=pd.read_table(file,dtype=str,sep="|")
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['sales_instore']=np.where(df['location_id']!="6990",df['item_transaction_amt'],0)
    df['sales_online']=np.where(df['location_id']=="6990",df['item_transaction_amt'],0)
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans['trans_instore']=np.where(df_trans['location_id']!="6990",1,0)
    df_trans['trans_online']=np.where(df_trans['location_id']=="6990",1,0)
    
    df_trans=df_trans.groupby(["customer_id_hashed","transaction_dt"])['trans_instore','trans_online'].sum().reset_index()
    
    df_sales=df.groupby(["customer_id_hashed","transaction_dt"])['sales_instore','sales_online'].sum().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed","transaction_dt"],how="outer")
    df_POS_all=df_POS_all.append(df)
    print(datetime.datetime.now(),df_POS_all.shape)
    
df_POS_all.shape
# earlier version doesn't devide the online vs instore

2019-12-11 17:30:15.780197 (1409778, 6)


In [ ]:
df_FB_id_email_all=df_FB_id_email_all[['customer_id_hashed','email_address_hash','Group','redemption','unsubscription','FB_clicker']]

In [ ]:
df_FB_id_email_all.head(4)

In [ ]:
P1_range=["2019-10-21","2019-11-02"]
P2_range=["2019-11-03","2019-11-07"]

In [ ]:
print(df_POS_all['transaction_dt'].min(),df_POS_all['transaction_dt'].max())
print(df_FB_id_email_all.shape)
df_FB_id_email_all.head(2)

In [ ]:
df_FB_id_email_all.shape, df_FB_id_email_all['customer_id_hashed'].nunique()

In [ ]:
# left merge into the POS
df_POS_all=pd.merge(df_POS_all,df_FB_id_email_all,on="customer_id_hashed",how="left")
print(df_POS_all.shape)
df_POS_all.head(2)

In [ ]:
df_POS_all['Period']=np.where(((df_POS_all['transaction_dt']>=P1_range[0]) & (df_POS_all['transaction_dt']<=P1_range[1])),"P1: 10/21-11/2",
                              np.where(((df_POS_all['transaction_dt']>=P2_range[0]) & (df_POS_all['transaction_dt']<=P2_range[1])),"P2: 11/3-11/7","Others")
                             )

In [ ]:
df_POS_all.to_csv("./BL_df_detail_3_weeks_POS_instore_and_online_with_redeemers_subscribe_FBclicker_JL_"+str(datetime.datetime.now().date())+".csv",index="False")



In [ ]:
# Changed back to count customer_id (not email) because the 543 clickers

In [ ]:
agg_func={"customer_id_hashed":"nunique","sales_instore":"sum","sales_online":"sum","trans_instore":"sum","trans_online":"sum"}
df_summary=df_POS_all.groupby(["Group","Period","redemption","unsubscription","FB_clicker"]).agg(agg_func).reset_index()
df_summary.shape

In [ ]:
df_total_id=df_FB_id_email_all.groupby(["Group","redemption","unsubscription","FB_clicker"])['customer_id_hashed'].nunique().to_frame().reset_index()
df_total_id

In [ ]:
df_unique_shoppers_both_periods=df_POS_all[df_POS_all['Period']!="Others"]
df_unique_shoppers_both_periods=df_unique_shoppers_both_periods.groupby(["Group","redemption","unsubscription","FB_clicker"])['customer_id_hashed'].nunique().to_frame().reset_index()
df_unique_shoppers_both_periods.shape

In [ ]:
writer=pd.ExcelWriter("./BL_output_FB_offer_result_by_store_type_redemption_unsubscription_clickers_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary.to_excel(writer,"performance",index=False)
df_total_id.to_excel(writer,"total_id",index=False)
df_unique_shoppers_both_periods.to_excel(writer,"total_shoppers_both_P",index=False)
writer.save()